In [9]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import pyspark
import numpy as np
import pyspark.sql.functions as sf
import matplotlib.pyplot as plt
%matplotlib inline


import logging
logging.basicConfig(level=logging.INFO)

# Create spark session
spark = (pyspark.sql.SparkSession.builder.getOrCreate())

sc = spark.sparkContext

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from pyspark.sql.dataframe import DataFrame

def pipe(df, f, *args, **kwargs):
    return f(df, *args, **kwargs)

DataFrame.pipe = pipe

In [137]:
from formula1 import pyspark_preprocessing

In [138]:
work_df = (
    pyspark_preprocessing.read_data()
    .pipe(pyspark_preprocessing.join_constructors)
    .pipe(pyspark_preprocessing.join_drivers)
    .pipe(pyspark_preprocessing.join_races)
    .pipe(pyspark_preprocessing.join_status)
    .pipe(pyspark_preprocessing.drop_columns)
    .pipe(pyspark_preprocessing.sort_races)
)

In [139]:
work_df.show()

+--------+------+--------+-------------+------+----+--------+-------------+------+----+------------+----------+----+--------------+---------------+--------------+-----------+-----------+--------------+------+--------+--------------+-----------+----+-----+---------+------------------+-------------------+--------------+
|statusId|raceId|driverId|constructorId|number|grid|position|positionOrder|points|laps|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|constructorRef|       name|nationality|     driverRef|number|forename|       surname|nationality|year|round|circuitId|              name|               date|        status|
+--------+------+--------+-------------+------+----+--------+-------------+------+----+------------+----------+----+--------------+---------------+--------------+-----------+-----------+--------------+------+--------+--------------+-----------+----+-----+---------+------------------+-------------------+--------------+
|       1|   833|     642|           51|

In [118]:
work_df.columns

['statusId',
 'raceId',
 'driverId',
 'constructorId',
 'number',
 'grid',
 'position',
 'positionOrder',
 'points',
 'laps',
 'milliseconds',
 'fastestLap',
 'rank',
 'fastestLapTime',
 'fastestLapSpeed',
 'constructorRef',
 'name',
 'nationality',
 'driverRef',
 'number',
 'forename',
 'surname',
 'nationality',
 'year',
 'round',
 'circuitId',
 'name',
 'date',
 'status']

In [ ]:
@log_step
def average_finishing(df):
    return df.assign(
        mean_position_till_date = lambda df: df.groupby('driverId')['positionOrder']
        .transform(lambda df: df.shift(1).expanding().mean())
    )

In [140]:
from pyspark.sql.window import Window

In [189]:
def diff_between_races(df):
    window = Window.partitionBy('driverId').orderBy('raceId')
    
    return df.withColumn('difference_races', sf.col('positionOrder') - sf.lag(sf.col('positionOrder')).over(window))

In [190]:
x = diff_between_races(work_df)

In [193]:
x.show()

+--------+------+--------+-------------+------+----+--------+-------------+------+----+------------+----------+----+--------------+---------------+--------------+---------+-----------+---------+------+--------+-------+-----------+----+-----+---------+--------------------+-------------------+------------------+----------------+
|statusId|raceId|driverId|constructorId|number|grid|position|positionOrder|points|laps|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|constructorRef|     name|nationality|driverRef|number|forename|surname|nationality|year|round|circuitId|                name|               date|            status|difference_races|
+--------+------+--------+-------------+------+----+--------+-------------+------+----+------------+----------+----+--------------+---------------+--------------+---------+-----------+---------+------+--------+-------+-----------+----+-----+---------+--------------------+-------------------+------------------+----------------+
|       3|   

In [184]:
def average_finishing(df):
    window =  Window.partitionBy('driverId')
    
    return df.withColumn('mean_position_till_date', sf.avg('positionOrder').over(window))

In [ ]:
sf.lag('temperature').over(mid_window))

In [183]:
work_df.show()

+--------+------+--------+-------------+------+----+--------+-------------+------+----+------------+----------+----+--------------+---------------+--------------+-----------+-----------+--------------+------+--------+--------------+-----------+----+-----+---------+------------------+-------------------+--------------+
|statusId|raceId|driverId|constructorId|number|grid|position|positionOrder|points|laps|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|constructorRef|       name|nationality|     driverRef|number|forename|       surname|nationality|year|round|circuitId|              name|               date|        status|
+--------+------+--------+-------------+------+----+--------+-------------+------+----+------------+----------+----+--------------+---------------+--------------+-----------+-----------+--------------+------+--------+--------------+-----------+----+-----+---------+------------------+-------------------+--------------+
|       1|   833|     642|           51|

In [180]:
def average_finishing_test(df):
    window =  Window.partitionBy('driverId')
    
    return df.withColumn('lagged_by_one', sf.lag('positionOrder').over(window))

In [181]:
x = average_finishing_test(work_df)

AnalysisException: 'Window function lag(positionOrder#6326, 1, null) requires window to be ordered, please add ORDER BY clause. For example SELECT lag(positionOrder#6326, 1, null)(value_expr) OVER (PARTITION BY window_partition ORDER BY window_ordering) from table;'

In [182]:
x.show()

Py4JJavaError: An error occurred while calling o1570.showString.
: java.lang.UnsupportedOperationException: Cannot evaluate expression: lag(input[7, int, true], 1, null)
	at org.apache.spark.sql.catalyst.expressions.Unevaluable$class.eval(Expression.scala:258)
	at org.apache.spark.sql.catalyst.expressions.OffsetWindowFunction.eval(windowExpressions.scala:337)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.eval(Expression.scala:389)
	at org.apache.spark.sql.catalyst.expressions.Alias.eval(namedExpressions.scala:152)
	at org.apache.spark.sql.catalyst.expressions.InterpretedUnsafeProjection.apply(InterpretedUnsafeProjection.scala:76)
	at org.apache.spark.sql.execution.TakeOrderedAndProjectExec$$anonfun$executeCollect$1.apply(limit.scala:139)
	at org.apache.spark.sql.execution.TakeOrderedAndProjectExec$$anonfun$executeCollect$1.apply(limit.scala:139)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$ano

In [ ]:
df.filter(df.X == 1).toPandas()

In [163]:
x.filter(x.driverRef == 'hamilton').select('mean_position').show()

+-----------------+
|    mean_position|
+-----------------+
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
|5.485576923076923|
+-----------------+
only showing top 20 rows



In [ ]:
@log_step
def average_finishing(df):
    return df.assign(
        mean_position_till_date = lambda df: df.groupby('driverId')['positionOrder']
        .transform(lambda df: df.shift(1).expanding().mean())
    )

# Get the driver with the max driverID

In [38]:
work_df.select('driverRef','driverId').toPandas()

,driverRef,driverId
0,hamilton,1
1,heidfeld,2
2,rosberg,3
3,alonso,4
4,kovalainen,5
5,nakajima,6
6,bourdais,7
7,raikkonen,8
8,kubica,9
9,glock,10


In [44]:
max_driverId = work_df.agg({"driverId": "max"}).collect()[0][0]
max_driverId

843

In [70]:
work_df.filter(work_df.driverId == max_driverId).show()

+--------+------+--------+-------------+--------+------+----+--------+------------+-------------+------+----+----+------------+----------+----+--------------+---------------+--------------+----------+-----------+--------------------+----+---------------+------+----+--------+-------+----------+-------------+--------------------+----+-----+---------+--------------------+-------------------+--------+--------------------+------+
|statusId|raceId|driverId|constructorId|resultId|number|grid|position|positionText|positionOrder|points|laps|time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|constructorRef|      name|nationality|                 url| _c5|      driverRef|number|code|forename|surname|       dob|  nationality|                 url|year|round|circuitId|                name|               date|    time|                 url|status|
+--------+------+--------+-------------+--------+------+----+--------+------------+-------------+------+----+----+------------+----------+----

In [53]:
max_driver_RDD = work_df.filter(work_df.driverId == max_driverId)

In [69]:
max_driver_RDD.select('driverRef','driverId').collect()[0][0]

'brendon_hartley'